<a href="https://colab.research.google.com/github/KBVKarthik/Keras-Computer-Vision-Models/blob/main/Knowledge_Distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [45]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):

        x, y = data


        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:

            student_predictions = self.student(x, training=True)


            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss


        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)


        self.optimizer.apply_gradients(zip(gradients, trainable_vars))


        self.compiled_metrics.update_state(y, student_predictions)


        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):

        x, y = data


        y_prediction = self.student(x, training=False)


        student_loss = self.student_loss_fn(y, y_prediction)


        self.compiled_metrics.update_state(y, y_prediction)


        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [46]:
teacher = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(512, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="teacher",
)


student = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="student",
)


student_scratch = keras.models.clone_model(student)

In [47]:
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()


x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))

11493376/11490434 [==============================] - 0s 0us/step


In [48]:
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)


teacher.fit(x_train, y_train, epochs=5)
teacher.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 [==============================] - 32s 17ms/step - loss: 0.1411 - sparse_categorical_accuracy: 0.9574
Epoch 2/5
1875/1875 [==============================] - 31s 17ms/step - loss: 0.0910 - sparse_categorical_accuracy: 0.9725
Epoch 3/5
1875/1875 [==============================] - 31s 17ms/step - loss: 0.0809 - sparse_categorical_accuracy: 0.9765
Epoch 4/5
1875/1875 [==============================] - 31s 17ms/step - loss: 0.0743 - sparse_categorical_accuracy: 0.9788
Epoch 5/5
313/313 [==============================] - 2s 6ms/step - loss: 0.1056 - sparse_categorical_accuracy: 0.9741


[0.10561341792345047, 0.9740999937057495]

In [49]:
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)


distiller.fit(x_train, y_train, epochs=3)


distiller.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 [==============================] - 14s 7ms/step - sparse_categorical_accuracy: 0.9216 - student_loss: 0.3318 - distillation_loss: 0.0937
Epoch 2/3
1875/1875 [==============================] - 13s 7ms/step - sparse_categorical_accuracy: 0.9660 - student_loss: 0.1411 - distillation_loss: 0.0331
Epoch 3/3
313/313 [==============================] - 1s 3ms/step - sparse_categorical_accuracy: 0.9752 - student_loss: 0.0934


[0.9751999974250793, 0.10585574805736542]

In [50]:
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)


student_scratch.fit(x_train, y_train, epochs=3)
student_scratch.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2200 - sparse_categorical_accuracy: 0.9352
Epoch 2/3
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0835 - sparse_categorical_accuracy: 0.9749
Epoch 3/3
313/313 [==============================] - 1s 3ms/step - loss: 0.0540 - sparse_categorical_accuracy: 0.9827


[0.05402543768286705, 0.982699990272522]